In [43]:
# Remove repo in local Colab storage if needed
!rm -r Differential-Equation-Solutions-with-Multilayer-Perceptron

In [44]:
# clone repo if we don't have in this session, specify branch after -b
!git clone -b main https://github.com/thesullivantage/Differential-Equation-Solutions-with-Multilayer-Perceptron
# if already exists, just pull in next cell

Cloning into 'Differential-Equation-Solutions-with-Multilayer-Perceptron'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 121 (delta 58), reused 91 (delta 36), pack-reused 0
Receiving objects: 100% (121/121), 27.67 KiB | 3.46 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [45]:
# Update (pull) repo from branch of our choosing
!cd Differential-Equation-Solutions-with-Multilayer-Perceptron && git checkout main && git pull

Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.


In [46]:
import sys
import os
from importlib import reload

In [47]:
sys.path.insert(0, 'Differential-Equation-Solutions-with-Multilayer-Perceptron')

In [48]:
# to reload modules as we make changes
reload(loss)
reload(model)


<module 'model_pk.model' from '/content/Differential-Equation-Solutions-with-Multilayer-Perceptron/model_pk/model.py'>

In [49]:
import model_pk
from model_pk import loss, model

In [50]:
import sys
import tensorflow.keras as keras
import tensorflow as tf

In [51]:
# Range of independent variable values
start = 1 # Start value
limit = 100  # End value (exclusive)
delta = 0.1  # Step size

In [52]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
mlp = model.MLP(num_embed_layer=2, phys_dimension=1,embed_dim=32)
loss_fn = loss.LossODE(f0=0, dell=1e-4)


In [54]:
# Make simple ODE domain for f'(x) = 2x: STANDARDIZED OR NORMALIZED
phys_dimension = 1
# Range of independent variable values
start = 0  # Start value
limit = 1  # End value (exclusive)
delta = 0.01  # Step size

# Create a TensorFlow array in the style of np.arange
inputs = tf.reshape(tf.range(start, limit, delta, dtype=tf.float32), (-1, phys_dimension))


In [57]:
# Training loop
num_epochs = 25  # Define the number of epochs

### Trained using SGD optimization (MSE), without replacement
### n (dim. inputs) gradient updates
for epoch in range(1, num_epochs+1):
    ### increment squared loss over entire epoch (dataset)
    epoch_total_loss = 0
    for input_example in standardized_inputs:
        # seems easiest to resize individual data points
        input_example_reshaped = tf.reshape(input_example, (1,1))
        with tf.GradientTape() as tape:
            # Compute the loss for the current input example (squared error)
            loss_curr = loss_fn.compute_loss_element(mlp, input_example_reshaped)
            epoch_total_loss += loss_curr
        # Compute gradients and update model parameters
        gradients = tape.gradient(loss_curr, mlp.trainable_variables)
        optimizer.apply_gradients(zip(gradients, mlp.trainable_variables))
    epoch_avg_loss = epoch_total_loss / tf.cast(inputs.shape[0], tf.float32)
    print(f'Epoch {epoch} Average Loss: ', epoch_avg_loss[0][0])

Epoch 1 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 2 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 3 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 4 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 5 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 6 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 7 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 8 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 9 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 10 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 11 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 12 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 13 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 14 Average Loss:  tf.Tensor(23.991392, shape=(), dtype=float32)
Epoch 15 Average Loss:  tf.Te